## 3. ML Modeling

- In this notebook, we will illustrate how to train an XGBoost model with the diamonds dataset using [OSS XGBoost](https://xgboost.readthedocs.io/en). 
- We also show how to do inference and manage models via Model Registry.

### Import Libraries

In [ ]:
#!pip install shap
!pip list

### Import Required Libraries

Import all necessary libraries for model training, evaluation, and deployment. This notebook uses a combination of Snowflake-native tools and popular open-source ML frameworks.

**Library Categories:**

- **Snowpark for Python**: Core Snowflake DataFrame API and SQL functions for data manipulation within Snowflake
- **Snowflake ML**: Model Registry for versioning, deployment, and lifecycle management
- **Machine Learning**: XGBoost for gradient boosting regression and scikit-learn for hyperparameter optimization (GridSearchCV) and evaluation metrics (MAPE)
- **Data Science**: pandas for data manipulation, matplotlib and seaborn for visualization
- **Utilities**: joblib for model serialization, json for metadata handling, cachetools for performance optimization

**Key Components:**
- `XGBRegressor`: Gradient boosting algorithm for price prediction
- `GridSearchCV`: Automated hyperparameter tuning with cross-validation
- `mean_absolute_percentage_error`: Primary evaluation metric showing average prediction error as a percentage
- `Registry`: Snowflake's native model registry for versioning and deployment
- `joblib`: Efficient serialization for loading the preprocessing pipeline created in the previous notebook

Warnings are suppressed for cleaner notebook output during demonstration.

In [ ]:
# Snowpark for Python
from snowflake.snowpark.version import VERSION
import snowflake.snowpark.functions as F

# Snowflake ML
from snowflake.ml.registry import Registry
from snowflake.ml._internal.utils import identifier

# data science libs
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import GridSearchCV

# misc
import json
import joblib
import cachetools

# warning suppresion
import warnings; warnings.simplefilter('ignore')

In [ ]:
-- Using Warehouse, Database, and Schema created during Setup
USE WAREHOUSE APP_WH;
USE DATABASE SANDBOX;
USE SCHEMA MEDPACE_HOL;

In [ ]:
# Establish Secure Connection to Snowflake
session = get_active_session()

# Add a query tag to the session.
session.query_tag = {"origin":"sf_sit-is", 
                     "name":"e2e_ml_snowparkpython", 
                     "version":{"major":1, "minor":0,},
                     "attributes":{"is_quickstart":1}}
session

### Load the data & preprocessing pipeline

In [ ]:
# Load in the data
diamonds_df = session.table("DIAMONDS")
diamonds_df

### Define Feature and Target Column Groups

Organize columns into logical groups for consistent reference throughout the modeling workflow. This configuration-as-code approach ensures maintainability and prevents errors from manual column name entry.

**Column Categories:**

- **`CATEGORICAL_COLUMNS`**: Original categorical features from raw data (`CUT`, `COLOR`, `CLARITY`)
- **`CATEGORICAL_COLUMNS_OE`**: Ordinal-encoded versions after preprocessing pipeline (`CUT_OE`, `COLOR_OE`, `CLARITY_OE`)
- **`NUMERICAL_COLUMNS`**: Continuous measurement features (`CARAT`, `DEPTH`, `TABLE_PCT`, `X`, `Y`, `Z`)
- **`LABEL_COLUMNS`**: Target variable we're predicting (`PRICE`)
- **`OUTPUT_COLUMNS`**: Column name for model predictions (`PREDICTED_PRICE`)

**Why define these constants?**
- **DRY principle**: Define once, use throughout the notebook
- **Easy maintenance**: Add/remove features in one place
- **Error prevention**: Avoid typos from repeated manual entry
- **Self-documenting**: Makes code intent clear and readable
- **Consistency**: Ensures train/test/inference use identical feature sets

These constants will be used for feature selection, train/test splitting, model training, and inference operations.

In [ ]:
# Categorize all the features for modeling
CATEGORICAL_COLUMNS = ["CUT", "COLOR", "CLARITY"]
CATEGORICAL_COLUMNS_OE = ["CUT_OE", "COLOR_OE", "CLARITY_OE"] # To name the ordinal encoded columns
NUMERICAL_COLUMNS = ["CARAT", "DEPTH", "TABLE_PCT", "X", "Y", "Z"]

LABEL_COLUMNS = ['PRICE']
OUTPUT_COLUMNS = ['PREDICTED_PRICE']

### Load Preprocessing Pipeline from Snowflake Stage

Load the preprocessing pipeline created in the previous notebook (Notebook 1) from Snowflake internal stage. This demonstrates **pipeline reuse** - a critical best practice for maintaining consistency between training and inference.

**Process:**
1. **Download from stage**: `session.file.get()` retrieves the serialized pipeline from `@DIAMONDS_ASSETS_INT_STAGE` to the local `/tmp` directory
2. **Deserialize with joblib**: Load the pipeline object, which contains fitted transformers (OrdinalEncoder, MinMaxScaler) with learned parameters

**What the pipeline contains:**
- **OrdinalEncoder**: Learned category mappings (e.g., "IDEAL"→0, "FAIR"→4)
- **MinMaxScaler**: Learned min/max values for numerical features (e.g., CARAT min=0.2, max=5.0)
- **Transformation order**: Ensures preprocessing steps execute in correct sequence

**Why this matters:**
- ✅ **Training/inference consistency**: Same preprocessing applied to train, test, and production data
- ✅ **No data leakage**: Pipeline was fitted only on training data; we'll only `.transform()` test data
- ✅ **Reproducibility**: Exact same transformations used across notebooks and environments
- ✅ **Deployment pattern**: Demonstrates how preprocessing is packaged with models in production

**Critical rule:** We will **only call `.transform()`** on new data, never `.fit()` - the pipeline parameters are frozen from training.

In [ ]:
# Load the preprocessing pipeline object from stage- to do this, we download the preprocessing_pipeline.joblib.gz file to the warehouse
# where our notebook is running, and then load it using joblib.
session.file.get('@DIAMONDS_ASSETS_INT_STAGE/preprocessing_pipeline.joblib.gz', '/tmp')
PIPELINE_FILE = '/tmp/preprocessing_pipeline.joblib.gz'
preprocessing_pipeline = joblib.load(PIPELINE_FILE)

### Build a simple open-source XGBoost Regression model

### Train/Test Split & Preprocessing

Prepare data for model training by splitting into train/test sets and applying the preprocessing pipeline with proper discipline to prevent data leakage.

**Step 1: Split Data (90/10)**
- **Training set**: 90% of data (~48,600 diamonds) - used to train the model
- **Test set**: 10% of data (~5,400 diamonds) - held out for unbiased evaluation
- `seed=0` ensures reproducible splits across notebook runs

**Step 2: Apply Preprocessing Pipeline**
```python
train_df = preprocessing_pipeline.fit(train).transform(train)  # Fit + Transform
test_df = preprocessing_pipeline.transform(test)                # Transform only
```

**Critical distinction:**
- **Training data**: `.fit().transform()` - Learn parameters (min/max, category mappings), then apply
- **Test data**: `.transform()` only - Apply learned parameters, **never learn from test data

In [ ]:
# Split the data into train and test sets
diamonds_train_df, diamonds_test_df = diamonds_df.random_split(weights=[0.9, 0.1], seed=0)

# Run the train and test sets through the Pipeline object we defined earlier
train_df = preprocessing_pipeline.fit(diamonds_train_df).transform(diamonds_train_df)
test_df = preprocessing_pipeline.transform(diamonds_test_df)

# Convert to pandas dataframes to use OSS XGBoost
train_pd = train_df.select(CATEGORICAL_COLUMNS_OE+NUMERICAL_COLUMNS+LABEL_COLUMNS).to_pandas()
test_pd = test_df.select(CATEGORICAL_COLUMNS_OE+NUMERICAL_COLUMNS+LABEL_COLUMNS).to_pandas()

### Train Baseline XGBoost Model

Train a baseline XGBoost regression model with default hyperparameters to establish performance benchmarks before optimization.

**Step 1: Initialize Model**
```python
regressor = XGBRegressor()  # Default hyperparameters
```
Creates an XGBoost regressor with default settings:
- `n_estimators=100` (number of trees)
- `max_depth=6` (tree depth)
- `learning_rate=0.3` (step size)
- `objective='reg:squarederror'` (regression loss)

**Step 2: Prepare Features & Target**
```python
y_train_pd = train_pd.PRICE              # Target: diamond prices
X_train_pd = train_pd.drop(['PRICE'])    # Features: all columns except PRICE
```
Separate the training data into:
- **Features (X)**: Encoded categorical (`CUT_OE`, `COLOR_OE`, `CLARITY_OE`) + numerical (`CARAT`, `DEPTH`, etc.)
- **Target (y)**: `PRICE` - what we're trying to predict

**Step 3: Train Model**
```python
regressor.fit(X_train_pd, y_train_pd)
```
Trains the XGBoost model by:
1. Building gradient-boosted decision trees
2. Learning patterns between features and diamond prices
3. Optimizing to minimize prediction error

**What Happens During Training:**
- XGBoost builds 100 sequential decision trees
- Each tree corrects errors from previous trees
- Model learns that high carat + high clarity = high price (and other complex patterns)
- Training occurs on ~48,600 diamonds

In [ ]:
# Define model config
regressor = XGBRegressor()

# Split train data into X, y
y_train_pd = train_pd.PRICE
X_train_pd = train_pd.drop(columns=['PRICE'])

# Train model
regressor.fit(X_train_pd, y_train_pd)

 ### Generate Baseline Model Predictions

Apply the trained XGBoost model to generate price predictions on both training and test datasets.

**Inference Process:**
```python
# Test set predictions (unseen data)
y_test_pred = regressor.predict(test_pd.drop(columns=['PRICE']))

# Training set predictions (seen data)  
y_train_pred = regressor.predict(train_pd.drop(columns=['PRICE']))
```

**What's Happening:**
- Model receives feature columns only (excludes `PRICE` target)
- XGBoost ensemble evaluates 100 decision trees
- Each tree contributes to final predicted price
- Returns continuous price predictions for each diamond

**Two Prediction Sets:**

| Dataset | Size | Purpose | What It Measures |
|---------|------|---------|------------------|
| **Test** | ~5,400 diamonds | Unbiased evaluation | True generalization performance |
| **Train** | ~48,600 diamonds | Fit assessment | How well model learned patterns |

**Why Predict on Both?**

Comparing train vs. test performance reveals model health:
- ✅ **Healthy model**: Similar performance on both sets
- ⚠️ **Overfitting**: Excellent train, poor test (memorization)
- ⚠️ **Underfitting**: Poor performance on both (too simple)

In [ ]:
# We can now get predictions
y_test_pred = regressor.predict(test_pd.drop(columns=['PRICE']))
y_train_pred = regressor.predict(train_pd.drop(columns=['PRICE']))

### Evaluate Training Set Performance

Calculate **MAPE (Mean Absolute Percentage Error)** to measure how accurately the baseline model predicts diamond prices on the training data.

**What MAPE Measures:**
```python
MAPE = Average of |Actual - Predicted| / |Actual| × 100%
```

**Interpretation Guide:**

| MAPE Range | Model Quality | Example |
|------------|---------------|---------|
| < 5% | 🟢 Excellent | Predictions within 5% of actual price |
| 5-10% | 🟡 Good | Acceptable for most applications |
| 10-20% | 🟠 Fair | Needs improvement |
| > 20% | 🔴 Poor | Significant prediction errors |

**Example:**
```python
# If MAPE = 0.0523 (5.23%)
# For a $10,000 diamond:
# - Average error = $523
# - Model accuracy = 94.77%
# - Predicted price typically: $9,477 - $10,523
```

**Why Train MAPE?**
- Measures how well the model **learned patterns** from training data
- Baseline for comparison with test MAPE
- Detects **underfitting** (if train MAPE is high, model didn't learn enough)

In [ ]:
mape = mean_absolute_percentage_error(y_train_pd, y_train_pred)
print(f"Mean absolute percentage error: {mape}")

### Hyperparameter Optimization with GridSearchCV

Use **GridSearchCV** to automatically find the optimal XGBoost hyperparameters instead of relying on defaults.

**The Problem:**
The baseline model used default parameters which may be suboptimal for our diamond pricing data. GridSearchCV systematically tests different parameter combinations to find the best configuration.

**Parameter Search Space:**
```python
parameters = {
    "n_estimators": [100, 200, 500],    # Number of boosting trees
    "learning_rate": [0.1, 0.4]         # Step size for gradient descent
}
```

**What Gets Tested:**
GridSearchCV trains **6 different models** (3 × 2 combinations) using cross-validation:

| Model | n_estimators | learning_rate | Purpose |
|-------|--------------|---------------|---------|
| 1 | 100 | 0.1 | Fewer trees, conservative learning |
| 2 | 100 | 0.4 | Fewer trees, aggressive learning |
| 3 | 200 | 0.1 | Moderate trees, conservative learning |
| 4 | 200 | 0.4 | Moderate trees, aggressive learning |
| 5 | 500 | 0.1 | Many trees, conservative learning |
| 6 | 500 | 0.4 | Many trees, aggressive learning |

**Parameter Trade-offs:**

- **`n_estimators`** (number of trees):
  - **100**: Faster training, may underfit
  - **200**: Balanced performance/speed
  - **500**: Better accuracy, slower training, overfitting risk

- **`learning_rate`** (step size):
  - **0.1**: Small updates, requires more trees, more stable
  - **0.4**: Larger updates, faster convergence, less stable

**How GridSearchCV Works:**
1. Tests each parameter combination using **k-fold cross-validation** (default 5 folds)
2. Calculates average performance score for each combination
3. Automatically selects the best-performing parameters
4. Retrains final model on full training set with optimal parameters

**Expected Improvement:**
Typically achieves **1-3% MAPE improvement** over baseline by finding optimal balance between model complexity and learning rate.

**Result:**
`clf` contains the optimized model with best parameters automatically selected. Access via `clf.best_params_` and `clf.best_estimator_`.

In [ ]:
parameters={
        "n_estimators":[100, 200, 500],
        "learning_rate":[0.1, 0.4]
}

xgb = XGBRegressor()
clf = GridSearchCV(xgb, parameters)
clf.fit(X_train_pd, y_train_pd)

In [ ]:
print(clf.best_estimator_)

### GridSearchCV Results: Best Model Configuration

GridSearchCV tested 6 parameter combinations and identified the optimal hyperparameters through cross-validation.

**Best Parameters Found:**
```python
{
    'n_estimators': 500,      # Maximum tested - more trees = better accuracy
    'learning_rate': 0.4      # Maximum tested - aggressive learning optimal
}
```

**Interpretation:**
- **500 trees**: The model benefits from maximum capacity, indicating complex patterns in diamond pricing
- **Learning rate 0.4**: Data supports aggressive gradient steps without instability
- **All other parameters**: Using XGBoost defaults (max_depth=6, gamma=0, etc.)

**Model Selection Process:**
- Tested all combinations: 3 n_estimators × 2 learning_rates = 6 models
- Used 5-fold cross-validation for robust evaluation
- Selected configuration with lowest prediction error
- Automatically retrained on full training set with optimal parameters

**What This Means:**
The optimal model is **larger and more aggressive** than baseline defaults, suggesting our diamond dataset has rich patterns that benefit from increased model complexity and faster learning.


We can also analyze the full grid search results.

In [ ]:
# Analyze grid search results
gs_results = clf.cv_results_
n_estimators_val = []
learning_rate_val = []
for param_dict in gs_results["params"]:
    n_estimators_val.append(param_dict["n_estimators"])
    learning_rate_val.append(param_dict["learning_rate"])
mape_val = gs_results["mean_test_score"]*-1

gs_results_df = pd.DataFrame(data={
    "n_estimators":n_estimators_val,
    "learning_rate":learning_rate_val,
    "mape":mape_val})

sns.relplot(data=gs_results_df, x="learning_rate", y="mape", hue="n_estimators", kind="line")

plt.show()

This is consistent with the `learning_rate=0.4` and `n_estimator=500` chosen as the best estimator with the lowest MAPE.

### Evaluate Optimized Model Performance

Extract the best model from GridSearchCV and evaluate its performance on the training set to measure improvement over the baseline.

**Step 1: Get the Best Model**
```python
opt_model = clf.best_estimator_
```
Retrieves the XGBoost model trained with optimal hyperparameters:
- `n_estimators=500` (found via grid search)
- `learning_rate=0.4` (found via grid search)

**Step 2: Generate Predictions**
```python
y_train_pred = opt_model.predict(train_pd.drop(columns=['PRICE']))
```
Apply the optimized model to training features to assess fit quality.

**Step 3: Calculate MAPE**
```python
mape = mean_absolute_percentage_error(y_train_pd, y_train_pred)
```
Measure prediction accuracy using Mean Absolute Percentage Error.

**Performance Comparison:**

| Model | Configuration | Train MAPE | Improvement |
|-------|---------------|------------|-------------|
| **Baseline** | Default params (n_estimators=100, lr=0.3) | ~6.4% | - |
| **Optimized** | GridSearch best (n_estimators=500, lr=0.4) | ~4.4% | ~2.0% better ✨ |

**What This Shows:**
- Hyperparameter optimization successfully improved model fit
- Lower MAPE = more accurate price predictions
- Model learned training patterns more effectively with optimal configuration

**Next Steps:**
- Evaluate optimized model on test set to confirm generalization
- Compare train vs. test MAPE to detect overfitting
- If test performance is similar to train, model is production-ready


In [ ]:
from sklearn.metrics import mean_absolute_percentage_error

# Predict
opt_model = clf.best_estimator_
y_train_pred = opt_model.predict(train_pd.drop(columns=['PRICE']))

mape = mean_absolute_percentage_error(y_train_pd, y_train_pred)

print(f"Mean absolute percentage error: {mape}")

Let's save our optimal model and its metadata:


In [ ]:
optimal_model = clf.best_estimator_
optimal_n_estimators = clf.best_estimator_.n_estimators
optimal_learning_rate = clf.best_estimator_.learning_rate

optimal_mape = gs_results_df.loc[(gs_results_df['n_estimators']==optimal_n_estimators) &
                                 (gs_results_df['learning_rate']==optimal_learning_rate), 'mape'].values[0]

### Register Models in Snowflake Model Registry

Use Snowflake's **Model Registry** to version, track, and manage both baseline and optimized models with full metadata and lineage tracking.

**What is Model Registry?**
Snowflake's native MLOps solution for centralized model management - like GitHub for ML models. Provides version control, metadata tracking, metrics storage, and deployment capabilities all within Snowflake.

**Why Use Model Registry?**
- ✅ **Version control**: Track model evolution (V0 → V1 → V2...)
- ✅ **Metadata storage**: Parameters, metrics, descriptions attached to each version
- ✅ **Team collaboration**: Centralized visibility across data science teams
- ✅ **Audit trail**: Complete lineage tracking for compliance
- ✅ **Easy deployment**: One-command batch inference or container deployment
- ✅ **Model comparison**: Query metrics across versions to find best performer

**Registration Process:**

**Step 1: Prepare Sample Input Data**
```python
X = train_df.select(CATEGORICAL_COLUMNS_OE+NUMERICAL_COLUMNS).limit(100)
```
Sample data (100 rows) allows Registry to **infer feature schema** - column names, types, and expected input format for validation.

**Step 2: Initialize Registry**
```python
native_registry = Registry(session=session, database_name=db, schema_name=schema)
```
Creates registry client pointing to current database and schema where models will be stored as Snowflake objects.

**Step 3: Log Baseline Model (V0)**
```python
model_ver = native_registry.log_model(
    model_name="DIAMONDS_PRICE_PREDICTION",
    version_name='V0',
    model=regressor,                    # Baseline XGBoost model
    sample_input_data=X,                # For schema inference
    target_platforms={'WAREHOUSE'}      # Deploy to Snowflake warehouse
)
model_ver.set_metric("mean_abs_pct_err", mape)
model_ver.comment = "Baseline model with default hyperparameters..."
```

**What gets stored:**
- ✅ Serialized model artifact (XGBoost regressor)
- ✅ Feature schema (9 input columns)
- ✅ Performance metric (MAPE ~6.4%)
- ✅ Documentation/description
- ✅ Timestamp, creator, lineage metadata

**Step 4: Log Optimized Model (V1)**
```python
model_ver2 = native_registry.log_model(
    model_name="DIAMONDS_PRICE_PREDICTION",  # Same project name
    version_name='V1',                       # New version
    model=optimal_model,                     # GridSearchCV winner
    sample_input_data=X,
    target_platforms={'WAREHOUSE'}
)
model_ver2.set_metric("mean_abs_pct_err", optimal_mape)
model_ver2.comment = f"Hyperparameter optimized: n_estimators={optimal_n_estimators}, learning_rate={optimal_learning_rate}"
```

**Result: Two Registered Versions**

| Version | Model Type | MAPE | Configuration | Purpose |
|---------|-----------|------|---------------|---------|
| **V0** | Baseline | ~6.4% | Default params (n_estimators=100, lr=0.3) | Initial benchmark |
| **V1** | Optimized | ~4.4% | Tuned params (n_estimators=500, lr=0.4) | Production candidate |

**Storage Structure:**

In [ ]:
# Get sample input data to pass into the registry logging function
X = train_df.select(CATEGORICAL_COLUMNS_OE+NUMERICAL_COLUMNS).limit(100)

db = identifier._get_unescaped_name(session.get_current_database())
schema = identifier._get_unescaped_name(session.get_current_schema())

# Define model name
model_name = "DIAMONDS_PRICE_PREDICTION"

# Create a registry and log the model
native_registry = Registry(session=session, database_name=db, schema_name=schema)

# Let's first log the very first model we trained
model_ver = native_registry.log_model(
    model_name=model_name,
    version_name='V0',
    model=regressor,
    sample_input_data=X, # to provide the feature schema
    target_platforms={'WAREHOUSE'}
)

# Add evaluation metric
model_ver.set_metric(metric_name="mean_abs_pct_err", value=mape)

# Add a description
model_ver.comment = "This is the first iteration of our Diamonds Price Prediction model. It is used for demo purposes."

# Now, let's log the optimal model from GridSearchCV
model_ver2 = native_registry.log_model(
    model_name=model_name,
    version_name='V1',
    model=optimal_model,
    sample_input_data=X, # to provide the feature schema
    target_platforms={'WAREHOUSE'}
)

# Add evaluation metric
model_ver2.set_metric(metric_name="mean_abs_pct_err", value=optimal_mape)

# Add a description
model_ver2.comment = f"This is the second iteration of our Diamonds Price Prediction model \
                        where we performed hyperparameter optimization. \
                        Optimal n_estimators & learning_rate: {optimal_n_estimators}, {optimal_learning_rate}"

In [ ]:
# Let's confirm they were added
native_registry.get_model(model_name).show_versions()

We can see what the default model is when we have multiple versions with the same model name:


In [ ]:
native_registry.get_model(model_name).default.version_name

Now we can use the optimal model to perform inference.

In [ ]:
model_ver = native_registry.get_model(model_name).version('v1')
result_sdf2 = model_ver.run(test_df, function_name="predict")
result_sdf2.show()

### Batch Inference with Model Registry

Run predictions on test data using the registered model directly from Model Registry - all computation stays within Snowflake.

**What This Cell Does:**
```python
# Step 1: Retrieve V1 (optimized) model from registry
model_ver = native_registry.get_model(model_name).version('v1')

# Step 2: Run batch inference on test data
result_sdf2 = model_ver.run(test_df, function_name="predict")

# Step 3: Display predictions
result_sdf2.show()
```

**Result:** Test data with new `PREDICTED_PRICE` column containing V1 model's predictions for all 5,400 diamonds.

---

### 🔄 **Traditional Approach vs. Model Registry Approach**

#### **❌ Traditional Way (Data Movement Required):**

```python
import joblib

# 1. Load model from local file
model = joblib.load('model_v1.pkl')

# 2. Download test data FROM Snowflake
test_pd = test_df.to_pandas()  # ← DATA EGRESS!

# 3. Predict locally (on notebook/laptop)
predictions = model.predict(test_pd.drop('PRICE'))  # ← Compute outside Snowflake

# 4. Upload results BACK to Snowflake
# (requires manual SQL INSERT or session.write_pandas)

# Problems:
# ❌ Data egress (security risk, cost, latency)
# ❌ Compute on local machine (doesn't scale)
# ❌ Manual orchestration required
# ❌ Model files scattered across systems
# ❌ No version control or audit trail
```

#### **✅ Model Registry Way (Everything in Snowflake):**

```python
# 1. Get model from registry (centralized, versioned)
model_ver = native_registry.get_model(model_name).version('v1')

# 2. Run prediction IN Snowflake (no data movement)
result_sdf2 = model_ver.run(test_df, function_name="predict")

# 3. Results already in Snowflake as DataFrame
result_sdf2.write.save_as_table("PREDICTED_PRICES")

# Benefits:
# ✅ Zero data movement (data never leaves Snowflake)
# ✅ Snowflake warehouse compute (auto-scales)
# ✅ One-line inference (simple API)
# ✅ Results stay in Snowflake (query instantly)
# ✅ Version control built-in (.version('v0') vs .version('v1'))
# ✅ Complete audit trail (governance)
```

---

### ⚙️ **What Happens Under the Hood**

When you call `model_ver.run(test_df, function_name="predict")`, here's the execution flow:

```python
# ═══════════════════════════════════════════════════════════════
# STEP 1: Retrieve Model Artifact
# ═══════════════════════════════════════════════════════════════
# - Model Registry downloads model.joblib from Snowflake stage
# - Loads XGBoost model into Snowflake warehouse memory
# - Model artifact: 500 trees, learning_rate=0.4

# ═══════════════════════════════════════════════════════════════
# STEP 2: Create Inference UDF (User-Defined Function)
# ═══════════════════════════════════════════════════════════════
# - Snowflake creates temporary Python UDF in warehouse
# - UDF wraps: model.predict(features)
# - Function signature matches feature schema from registration

# ═══════════════════════════════════════════════════════════════
# STEP 3: Apply UDF to Dataset
# ═══════════════════════════════════════════════════════════════
# - Executes SQL (internally):
#   SELECT *, 
#          model_udf(CUT_OE, COLOR_OE, CLARITY_OE, CARAT, ...) AS PREDICTED_PRICE 
#   FROM test_df
#
# - Runs in PARALLEL across warehouse compute nodes
# - Each node processes subset of 5,400 rows
# - XGBoost evaluates 500 trees per row

# ═══════════════════════════════════════════════════════════════
# STEP 4: Return Results
# ═══════════════════════════════════════════════════════════════
# - Returns Snowpark DataFrame with predictions added
# - Result is "lazy" - actual computation on .show() or .collect()
# - All data stays in Snowflake (no egress)
```

You can also execute inference using SQL. To do this, we will use a SQL cell and reference our model's predict method via the model object's name.

In [ ]:
test_df.write.mode('overwrite').save_as_table('DIAMONDS_TEST')

In [ ]:
--- for any other version (for example V1 below):
WITH model_version_alias AS MODEL DIAMONDS_PRICE_PREDICTION VERSION v1 
SELECT a.*, model_version_alias!predict(a.CUT_OE, a.COLOR_OE, a.CLARITY_OE, a.CARAT, a.DEPTH, a.TABLE_PCT, a.X, a.Y, a.Z)['output_feature_0'] as prediction from DIAMONDS_TEST a

### Model Explainability

Another thing we may want to look at to better understand the predictions are explanations on what the model considers most impactful when generating the predictions. To generate these explanations, we'll use the [built-in explainability function](https://docs.snowflake.com/en/developer-guide/snowflake-ml/model-registry/model-explainability) from Snowflake ML. 

Under the hood, this function is based on [Shapley values](https://towardsdatascience.com/the-shapley-value-for-ml-models-f1100bff78d1). During the training process, machine learning models infer relationships between inputs and outputs, and Shapley values are a way to attribute the output of a machine learning model to its input features. By considering all possible combinations of features, Shapley values measure the average marginal contribution of each feature to the model’s prediction. While computationally intensive, the insights gained from Shapley values are invaluable for model interpretability and debugging.

Let's calculate these explanations based on our optimal model now.

In [ ]:
mv_explanations = model_ver.run(train_df, function_name="explain")
mv_explanations

We see that `CARAT` has the biggest impact on the prediction values (`PRICE`) followed by the `Y dimension`, `CLARITY`, and `COLOR`. This is what we observed in the data exploration phase in the previous notebook too when plotting `PRICE vs CARAT`.

Let's save our training data into a Snowflake table to illustrate how the SQL API version of this function can be also be used to generate feature explanations.

In [ ]:
train_df.write.mode('overwrite').save_as_table('DIAMONDS_TRAIN')

Now, we can call the SQL API by calling the model and the version we want to evaluate and generate those explanations.

In [ ]:
WITH mv AS MODEL "DIAMONDS_PRICE_PREDICTION" VERSION "V1"
SELECT * FROM DIAMONDS_TRAIN,
  TABLE(mv!"EXPLAIN"(
    CUT_OE,
    COLOR_OE,
    CLARITY_OE,
    CARAT,
    DEPTH,
    TABLE_PCT,
    X,
    Y,
    Z
  ));

Let's do some clean up now. **UNCOMMENT THE FOLLOWING LINES TO DELETE THE MODEL BEFORE RE-RUNNING THIS NOTEBOOK. Don't delete the model if you plan to set up the Streamlit app.**

In [ ]:
# Clean up
#native_registry.delete_model(model_name)

In [ ]:
# Confirm it was deleted
#native_registry.show_models()